Framework for making a input catalog...

1) read in a csv file of objects

2) call tess_ephem for each object to see if it has been observed by TESS

3) if it has, summarize the TESS info

4) output two files...  one lists all the bodies searched, the other is a catalog of the objects observed by TESS

feedback from protocatalog: (12/14)

+ add first time, last time on CCD.  preferably in TESS JD (might be JD - 2457000, but need to check...)

In [ ]:
from tess_ephem import ephem
import numpy as np
from astropy.time import Time
import pandas as pd
from itertools import repeat
from datetime import datetime

Starting with a list made by doing a search in Horizons small body search interface  (https://ssd.jpl.nasa.gov/sbdb_query.cgi). I don't see any documentation for an API, so you have to select the outputs manually.  To start, I've got the following columns:  PDE, full_name, a, e, i om, w, q, ad, per_y, condition_code, H, rot_per, class.  Whether or not these are the right columns is totally tbd!  Note, currently as long as the input list as a column for primary designation (pdes), the code will work.  The output catalog will just copy over any additional input columns into the output catalog

The current catalog returns a line for every unique combination of asteroid and sector.  (In otherwords, if an asteroid is present in multiple sectors, it will have multiple lines.)

Things to check -- do we need to do the same for cameras?  (Right now the code largely assumes that there is only one camera involved, but it does output the number of cameras (and ccds), so we can check this assumption and revisit later on.

In [71]:
# Want to see how long this all takes...
now = datetime.now().time() # time object
print("now =", now)

now = 10:37:58.609722


In [59]:
asteroid_list = pd.read_csv("horizons_2020_12_15.csv") #may have come back to this.  Suspect the pdes col should be string, not integer as we get further in the list
#asteroid_list = pd.read_csv("shortishlist.csv")
num_asteroids = asteroid_list.shape[0]
num_asteroids

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,10,17,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


1042723

In [60]:
asteroid_list.columns

Index(['pdes', 'full_name', 'a', 'e', 'i', 'om', 'w', 'q', 'ad', 'per_y',
       'condition_code', 'class', 'H', 'H_sigma', 'G', 'diameter',
       'diameter_sigma', 'extent', 'albedo', 'rot_per', 'spec_B', 'spec_T'],
      dtype='object')

In [61]:
# first off, a function that takes in tess_ephem output, asteroid info, and returns a one row data frame with results

def make_catalog_entry(ephem, asteroid):
    #calc values of interest
    num_of_days = ephem.shape[0]
    asteroid["num_of_days"] = num_of_days
    asteroid["sector"] = ephem.sector[0]
    asteroid["num_cameras"] = ephem.camera.nunique()
    asteroid["camera"] = ephem.camera[0]
    asteroid["num_ccd"] = ephem.ccd.nunique()
    asteroid["first_time"] = Time(ephem.index).jd[0] - 2457000  #constant is to conver to "TESS JD"
    asteroid["first_ccd"] = ephem.ccd[0]
    asteroid["first_column"] = ephem.column[0]
    asteroid["first_row"] = ephem.row[0]
    asteroid["last_time"] = Time(ephem.index).jd[-1] - 2457000
    asteroid["last_ccd"] = ephem.ccd[-1]
    asteroid["last_column"]  = ephem.column[(num_of_days-1)]
    asteroid["last_row"]= ephem.row[(num_of_days-1)]
    asteroid["max_pix_per_hour"] = ephem.pixels_per_hour.max()
    asteroid["min_Vmag"] = ephem.vmag.min()
    asteroid["mean_Vmag"] = ephem.vmag.mean()
    asteroid["max_Vmag"] = ephem.vmag.max()
    asteroid["min_phase_angle"] = ephem.phase_angle.min()
    asteroid["mean_phase_angle"] = ephem.phase_angle.mean()
    asteroid["max_phase_angle"] = ephem.phase_angle.max()
    asteroid["min_sun_distance"] = ephem.sun_distance.min()
    asteroid["mean_sun_distance"] = ephem.sun_distance.mean()
    asteroid["max_sun_distance"] = ephem.sun_distance.max()
    asteroid["min_tess_distance"] = ephem.obs_distance.min()
    asteroid["mean_tess_distance"] = ephem.obs_distance.mean()
    asteroid["max_tess_distance"] = ephem.obs_distance.max()
    
    return asteroid

there are two dataframes modified / created by this loop...

the original asteroid list will have a column added for whether or not the object is seen by tess.  this data frame includes a row for every asteroid we query about.

the output catalog will have an entry for each unique combination of asteroid, sector for every object seen by tess.  this data frame only has rows for objects seen by tess.

In [74]:
#now a version that results in multiple entries if there are multiple sectors
asteroid_catalog = []

false_list = []
false_list.extend(repeat(False, num_asteroids))
asteroid_list["in_Tess"] = false_list

starting_row = 52680 #set to zero for standard use.  set to other values to restart...

for idx, ad in asteroid_list[starting_row:].iterrows(): 
    tt = ephem(str(ad["pdes"]), verbose=True)
    #print(asteroid["pdes"], tt.shape[0])
    if tt.shape[0] == 0:
        continue
    #ad["in_Tess"] = True  #dunno why, but this doesn't modify the value in asteroid_list
    asteroid_list.loc[idx,"in_Tess"] = True
    #this_asteroid = ad.copy() #this works, but only if i use the ad["in_Tess"] = True
    this_asteroid = asteroid_list.loc[idx].copy()
    this_asteroid["num_sectors"] = tt.sector.nunique()
    for sector in tt.sector.unique():
        this_sector = tt.loc[tt["sector"] == sector]
        new_entry = make_catalog_entry(this_sector, this_asteroid.copy())
        asteroid_catalog.append(new_entry)
 
                           
asteroid_catalog = pd.DataFrame(asteroid_catalog)

ConnectionError: HTTPSConnectionPool(host='ssd.jpl.nasa.gov', port=443): Max retries exceeded with url: /horizons_batch.cgi?batch=1&TABLE_TYPE=OBSERVER&QUANTITIES=%271%2C3%2C9%2C19%2C20%2C43%27&COMMAND=%2252681%3B%22&SOLAR_ELONG=%220%2C180%22&LHA_CUTOFF=0&CSV_FORMAT=YES&CAL_FORMAT=BOTH&ANG_FORMAT=DEG&APPARENT=AIRLESS&REF_SYSTEM=J2000&EXTRA_PREC=NO&CENTER=%27%40TESS%27&START_TIME=%222018-07-25+00%3A00%3A00.000%22&STOP_TIME=%222020-08-25+00%3A00%3A00.000%22&STEP_SIZE=%227D%22&SKIP_DAYLT=NO (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb8e87765e0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [75]:
idx

52680

In [76]:
#this saves the ones taht successfully ran...
catalog_to_52680 = pd.DataFrame(asteroid_catalog)

In [77]:
catalog_to_25869.to_csv("catalog_to_52680.csv", index=False)

In [79]:
asteroid_list.pdes[idx]

52681

In [80]:
ephem("52681", verbose=True)

,sector,camera,ccd,column,row,pixels_per_hour,ra,dec,vmag,sun_distance,obs_distance,phase_angle
time,,,,,,,,,,,,


In [ ]:
# Want to see how long this all takes...
now = datetime.now().time() # time object
print("now =", now)

In [ ]:
asteroid_catalog.head()


In [ ]:
asteroid_list.head()

In [ ]:
asteroid_catalog.to_csv("TAC_2020_12_15.csv", index=False)

In [ ]:
asteroid_list.to_csv("asteroid_list_2020_12_15.csv",index=False)